<a href="https://colab.research.google.com/github/jso4342/STUDY__python/blob/main/convert_model_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import torch

In [36]:
from torch.utils.mobile_optimizer import optimize_for_mobile

In [37]:
import torchvision
import torchvision.models
import torchvision.models.segmentation

In [38]:
pth_model_path = '/content/model_mbv3_iou_mix_2C049.pth'
device = torch.device('cpu')

In [39]:
model = torch.load(pth_model_path, map_location=device)

In [40]:
model.eval()

AttributeError: ignored

In [ ]:
scriptedm = torch.jit.script(model)

In [ ]:
scriptedm.save("deeplabv3_scripted.pt")

AttributeError: ignored

In [ ]:
torch.jit.save(scriptedm, "torch_jit.pt")

AttributeError: ignored

In [ ]:
import torch
import torch.nn as nn

class TempModel(nn.Module):
    def __init__(self):
        self.conv1 = nn.Conv2d(3, 5, (3, 3))
    def forward(self, inp):
        return self.conv1(inp)

model = TempModel()
model.load_state_dict(torch.load(file_path))
model.eval()

In [ ]:
model.eval()

AttributeError: ignored

In [ ]:
scripted_module = torch.jit.script(model)

/usr/local/lib/python3.10/dist-packages/torch/jit/_recursive.py:260: UserWarning: 'aux_classifier' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn("'{}' was found in ScriptModule constants, "


In [ ]:
scripted_module.save("deeplabv3_scripted.pt")

In [ ]:
optimized_scripted_module = optimize_for_mobile(scripted_module)

In [ ]:
optimized_scripted_module._save_for_lite_interpreter("deeplabv3_scripted.ptl")

In [ ]:
# Export mobile interpreter version model (compatible with mobile interpreter)
optimized_scripted_module = optimize_for_mobile(scripted_module)
optimized_scripted_module._save_for_lite_interpreter("deeplabv3_scripted.ptl")

In [ ]:
import torch
from torch.utils.mobile_optimizer import optimize_for_mobile

# 편의를 위해 torch.hub에서 모델을 가져왔지만, pretrained 한 모델 또한 사용 가능
model = torch.hub.load('pytorch/vision:v0.7.0', 'deeplabv3_resnet50', pretrained=True)
model.eval()

scriptedm = torch.jit.script(model)
scriptedm.save("deeplabv3_scripted.pt") 	# 기존 pt 모델 저장
optimized_scripted_module = optimize_for_mobile(scriptedm)
optimized_scripted_module._save_for_lite_interpreter("deeplabv3_scripted_lite.ptl")

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.7.0
/usr/local/lib/python3.10/dist-packages/torch/jit/_recursive.py:260: UserWarning: 'aux_classifier' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn("'{}' was found in ScriptModule constants, "


In [ ]:
!pip install models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import torch
from models.generator import Generator
from torch.utils.mobile_optimizer import optimize_for_mobile


ModuleNotFoundError: ignored

In [ ]:
import torch
from models.generator import Generator
from torch.utils.mobile_optimizer import optimize_for_mobile

device = torch.device('cpu')



ModelName = "trained_netG.pth"
example_input = torch.rand(1, 3, 452, 340, dtype=torch.float)
model2  = Generator().to(device)
model2.load_state_dict(torch.load(ModelName, map_location=device))
model2.eval()
script_module = torch.jit.trace(model2.forward, example_input)


torchscript_model_optimized = optimize_for_mobile(script_module)
torch.jit.save(torchscript_model_optimized, "cartoon_test2.pt")

script_module.save('cartoon_test.pt')


loaded_model = torch.jit.load("cartoon_test.pt")
loaded_model.eval()
loaded_model.save('cartoon_test3.pt')

In [2]:
!pip install onnx_tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 37.8 MB/s eta 0:00:00


Convert ONNX model to TensorFlow format

In [10]:
from onnx_tf.backend import prepare
import torch.onnx
import onnx
import onnx_tf

In [12]:
onnx_model_path = "/content/deeplabv3_onnx_model.onnx"
onnx_model = onnx.load(onnx_model_path)
tf_rep = onnx_tf.backend.prepare(onnx_model)
tf_model_path = "/content/output/deeplabv3_tensorflow_from_onnx.pb"
tf_rep.export_graph(tf_model_path)

Convert Tensorflow model to Tensorflow Lite format

In [14]:
import tensorflow as tf

In [15]:
tflite_model_path = "deeplabv3_tensorflow_lite.tflite"
converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
tflite_model = converter.convert()
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)


양자화

In [24]:
import torch.quantization
import numpy as np

In [25]:
def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, 3, 384, 384)
      yield [data.astype(np.float32)]

In [28]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8   # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8

tflite_quant_model = converter.convert()

In [34]:
open("deeplanv3_tensorflow_quant_model.tflite", "wb").write(tflite_quant_model)

11607432